In [46]:
import pandas as pd
import numpy as np
import json
import shutil
import itertools

## Total number of trips per month

### per stop

In [59]:
dta = pd.read_csv('cfl 2017_2022_per month by year.csv', encoding='latin-1')

# transform
dta['date'] = pd.to_datetime(dta['mY'], format='%Y/%m')
dta.drop('mY', axis=1, inplace=True)
dta['monthly'] = dta[['Montee', 'Descente']].sum(axis=1)

# regroup by stop, year and month, ensuring there are 12 entries for every month
stats = dta.groupby(['LibBVU', dta.date.dt.year, dta.date.dt.month], observed=False).agg({'monthly': 'sum'})
stats = stats.rename_axis(['stop', 'year', 'month'])
monthly = stats.reindex(pd.MultiIndex.from_product(stats.index.levels)).groupby(level=[0, 1]).agg(list)

### per line

In [62]:
stats = dta.groupby(['Ligne', dta.date.dt.year, dta.date.dt.month], observed=False).agg({'monthly': 'sum'})
stats = stats.rename_axis(['stop', 'year', 'month'])
monthly_line = stats.reindex(pd.MultiIndex.from_product(stats.index.levels)).groupby(level=[0, 1]).agg(list)

## Average number of trips per day

### Per stop

In [64]:
stats = pd.read_csv('cfl 2017_2022_per day by year.csv', encoding='latin-1')
stats['daily'] = stats[['Montee_Weekday_vac', 'Descente_Weekday_vac']].sum(axis=1)
daily = stats.groupby(['LibBVU', 'Year']).agg({'daily': 'sum'}).round(0).astype(int)

### Per line

In [67]:
daily_line = stats.groupby(['Ligne', 'Year']).agg({'daily': 'sum'}).round(0).astype(int)

## Export to JSON

In [53]:
def to_nested_dict(dct):
    res = {}
    for keys, val in dct.items():
        target = res
        for key in keys:
            target = target.setdefault(key, {})
        for k, v in val.items():
            target[k] = [None if pd.isnull(s) else int(s) for s in v] if hasattr(v, '__iter__') else v
    return res

In [55]:
def merge_for_export(*args):
    dta = [a.to_dict(orient='index') for a in args]
    return {k: dict(itertools.chain(*(a[k].items() for a in dta))) for k in dta[0].keys()}
    

In [56]:
with open('trainstats.json', 'w', encoding='utf-8') as f:
    json.dump(to_nested_dict(merge_for_export(daily, monthly)), f)
shutil.copyfile('trainstats.json', '../../public/data/publictransport/trainstats.json')

'../../public/data/publictransport/trainstats.json'

In [69]:
with open('trainstats-line.json', 'w', encoding='utf-8') as f:
    json.dump(to_nested_dict(merge_for_export(daily_line, monthly_line)), f)
shutil.copyfile('trainstats-line.json', '../../public/data/publictransport/trainstats-line.json')

'../../public/data/publictransport/trainstats-line.json'